In [46]:
import numpy as np
import matplotlib.pyplot as plt

class Landmark:
    def __init__(self,x,y):
        self.position = np.array([x,y])
        
    def __str__(self):
        return "(%f,%f)" % (self.position[0],self.position[1]) 
    
    def x(self):
        return self.position[0]
    
    def y(self):
        return self.position[1]

        

In [47]:
actual_landmarks = (Landmark(0,0),Landmark(1,0),Landmark(0.5,0.5))

In [51]:
xs = [e.x() for e in actual_landmarks]
ys = [e.y() for e in actual_landmarks]

In [52]:
plt.scatter(xs,ys)